# Assignment 2 Part 2
### By: Kyle Jordan Deluz
This assignment showcases usage of Pandas to display and manipulate data, as well as doing basic CRUD using SQL statements.

In [33]:
# This imports the required libraries that will be used within this assignment.
import sqlite3
import pandas as pd
import uuid

In [34]:
# Generate a random database name using the UUID object
# Many types of UUID, but UUID4 will be the object that just sets a random set of characters.
# 1 would be based on hostID and time and 3 is MD5 hash of namespace UUID and a name
db_name = f'{uuid.uuid4().hex}.db'

In [35]:
# This then connects to the SQLite database
# This uses sqlite3 which is a module to work with SQLite in a Python environment, then using connect() connects it to the database name, which was referened above, based on the random name of the database.
# Finally, this creates a cursor object, which is a method of the conn object. The cursor is a handle that would interact with the database and execute SQL commands.
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

In [36]:
# Creates three tables using simple SQL statements.
# These tables then also contain 3 different rows that relate to that table, I am using an MMO that I play called Final Fantasy XIV, and in the game, they have character names, jobs, and job levels.
# The Retainers table includes a foreign key, which connects to the 2 other tables, in order to reference them when fetching data, or storing it.
cursor.execute('''
CREATE TABLE IF NOT EXISTS Players (
    PlayerID INTEGER PRIMARY KEY,
    PlayerName TEXT,
    Level INTEGER
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Jobs (
    JobID INTEGER PRIMARY KEY,
    JobName TEXT,
    Role TEXT
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Retainers (
    RetainerID INTEGER PRIMARY KEY,
    PlayerID INTEGER,
    RetainerName TEXT,
    JobID INTEGER,
    FOREIGN KEY(PlayerID) REFERENCES Players(PlayerID),
    FOREIGN KEY(JobID) REFERENCES Jobs(JobID)
)
''')

In [37]:
# This then inserts rows of data for each of the table, and includes the format of the ID, Name, Level/Job.
# For the retainers, since it's a table that contains foreign keys, it will connect which ID/Player it will be tied to, then add their information afterwards.
# It will output a Cursor at line, indicating that the cursor object is ready to interact with the database.
players_insert = [
    (1, 'Warrior of Light', 100), # WoL is Level 100
    (2, 'Thancred Waters', 90), # Thancred is Level 90
    (3, 'Alphinaud Leveilleur', 75) # Alphinaud is at level 75
]

jobs_insert = [
    (1, 'Miner', 'Gathering'), # The first job will be a miner, which is attached tothe Gathering category
    (2, 'Botanist', 'Gathering'), # Second is Botanist
    (3, 'Fisher', 'Gathering') # Third is Fisher
]

retainers_insert = [
    (1, 1, 'Tifa Lockhart', 1),  # WoL's retainer Tifa is a Miner
    (2, 2, 'Aerith Gainsborough', 2),   # Thancred's retainer Aerith is a Botanist
    (3, 3, 'Barret Wallace', 3)   # Alphinaud's retainer Barret is a Fisher
]

# Using executemany() is efficient when performing multiple executions of statements to the database.
# Handles bulk inserts or updates efficiently
cursor.executemany('INSERT INTO Players VALUES (?, ?, ?)', players_insert)
cursor.executemany('INSERT INTO Jobs VALUES (?, ?, ?)', jobs_insert)
cursor.executemany('INSERT INTO Retainers VALUES (?, ?, ?, ?)', retainers_insert)

In [38]:
# Commit the changes to the database and saves it.
conn.commit()

In [39]:
# This outputs everything about the table, by using the SELECT * condition in the SQL statement
# This will then print out the selected table within the database.
print("Players Table:")
players_df = pd.read_sql_query("SELECT * FROM Players", conn)
print(players_df)

print("\nJobs Table:")
jobs_df = pd.read_sql_query("SELECT * FROM Jobs", conn)
print(jobs_df)

print("\nRetainers Table:") 
retainers_df = pd.read_sql_query("SELECT * FROM Retainers", conn)
print(retainers_df)

Players Table:
   PlayerID            PlayerName  Level
0         1      Warrior of Light    100
1         2          Izumi Adachi     90
2         3  Alphinaud Leveilleur     75

Jobs Table:
   JobID   JobName       Role
0      1     Miner  Gathering
1      2  Botanist  Gathering
2      3    Fisher  Gathering

Retainers Table:
   RetainerID  PlayerID         RetainerName  JobID
0           1         1        Tifa Lockhart      1
1           2         2  Aerith Gainsborough      2
2           3         3       Barret Wallace      3


In [40]:
# This selects the table that needs to be updated, by using the cursor execute, it points to the Players table, and does the changes within the line
# Here, we set WoL's level to 95, instead of their given 100.
# Then we commit to the database.
cursor.execute('UPDATE Players SET Level = 95 WHERE PlayerID = 1')
conn.commit()

In [41]:
# After, we select the changed table and output the data within that table to the space below in Jupyter..
# This uses the Pandas module of read_sql_query which basically runs an SQL query, and saves it into the updated_player_df variable, which then prints it out.
print("\nUpdated Player Data (PlayerID = 1):")
updated_player_df = pd.read_sql_query("SELECT * FROM Players WHERE PlayerID = 1", conn)
print(updated_player_df)


Updated Player Data (PlayerID = 1):
   PlayerID        PlayerName  Level
0         1  Warrior of Light     95


In [42]:
# Deletes one row of data from the database that has the PlayerID of 2
# Same as before, it points to the table using the cursor, and executes the SQL statement line.
cursor.execute('DELETE FROM Players WHERE PlayerID = 2')
conn.commit()

In [43]:
# Output the entire table that the data was deleted from to the space below
print("\nPlayers Table after Deletion:")
players_df_after_deletion = pd.read_sql_query("SELECT * FROM Players", conn)
print(players_df_after_deletion)


Players Table after Deletion:
   PlayerID            PlayerName  Level
0         1      Warrior of Light     95
1         3  Alphinaud Leveilleur     75


In [44]:
# Close the connection to the database
conn.close()